In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import imageio.v2 as imageio   # use v2 to silence warning
from io import BytesIO
from tqdm import tqdm
import geopandas as gpd
import os

# ==============================
# User settings
# ==============================
CSV_FILE   = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\pasig_hourly_corrected.csv"
PASIG_SHP  = r"C:\Users\krish\Desktop\PhD Class\Shapefile\MM\Pasig\Pasig.shp"
OUT_DIR    = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\GIFs"
os.makedirs(OUT_DIR, exist_ok=True)

qa_mode = False          # True = show first 5 frames only, False = render GIF
months  = [1,2,3]           # choose which months to run (e.g. [1,2,3])

# AQI color bins (US EPA style)
aqi_bins   = [0, 12, 35.4, 55.4, 150.4, 250.4, 500]
aqi_colors = ["#00E400", "#FFFF00", "#FF7E00", "#FF0000", "#99004C", "#7E0023"]
aqi_labels = [
    "Good (0-12)",
    "Moderate (12.1-35.4)",
    "Unhealthy for Sensitive (35.5-55.4)",
    "Unhealthy (55.5-150.4)",
    "Very Unhealthy (150.5-250.4)",
    "Hazardous (250.5+)"
]

# ==============================
# Load data
# ==============================
df = pd.read_csv(CSV_FILE)

# combine Date + Time into datetime index
df["datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"])
df = df.set_index("datetime").sort_index()

# load Pasig shapefile
pasig = gpd.read_file(PASIG_SHP)
pasig_bounds = pasig.total_bounds  # xmin, ymin, xmax, ymax

# ==============================
# Helper: pick AQI color
# ==============================
def get_aqi_color(pm25):
    for i, limit in enumerate(aqi_bins[1:]):
        if pm25 <= limit:
            return aqi_colors[i]
    return aqi_colors[-1]

# ==============================
# Loop over months
# ==============================
for month in months:
    group = df[(df.index.year == 2025) & (df.index.month == month)]
    if group.empty:
        print(f"⚠️ No data found for 2025-{month:02d}")
        continue

    if qa_mode:
        print(f"📅 QA Preview for 2025-{month:02d} (first 5 frames)...")
        iter_group = group.groupby(group.index)
    else:
        out_gif = os.path.join(OUT_DIR, f"2025_{month:02d}.gif")
        writer = imageio.get_writer(out_gif, mode="I", duration=0.5)
        iter_group = tqdm(group.groupby(group.index), desc=f"Rendering 2025-{month:02d}")

    for i, (t, g) in enumerate(iter_group):
        if qa_mode and i >= 5:   # only preview first 5 frames
            break

        fig, ax = plt.subplots(figsize=(6, 6))

        # Pasig boundary (hairline style)
        pasig.boundary.plot(ax=ax, edgecolor="black", linewidth=0.3, zorder=1)

        # plot all stations at this timestamp
        for _, row in g.iterrows():
            ax.scatter(row["longitude"], row["latitude"],
                       s=120, c=get_aqi_color(row["pm25"]),
                       edgecolor="k", linewidth=0.5, zorder=2)
            ax.text(row["longitude"]+0.002, row["latitude"]+0.002,
                    f"{row['pm25']:.1f}", fontsize=6, zorder=3)

        ax.set_title(f"PM2.5 AQI - {t.strftime('%Y-%m-%d %H:%M')}", fontsize=12)
        ax.set_xlim(pasig_bounds[0]-0.01, pasig_bounds[2]+0.01)
        ax.set_ylim(pasig_bounds[1]-0.01, pasig_bounds[3]+0.01)

        # legend patches
        patches = [mpatches.Patch(color=aqi_colors[j], label=aqi_labels[j])
                   for j in range(len(aqi_labels))]
        ax.legend(handles=patches, loc="lower left", fontsize=6, frameon=True)

        if qa_mode:
            plt.show()
        else:
            buf = BytesIO()
            plt.savefig(buf, format="png", dpi=80, bbox_inches="tight")
            buf.seek(0)
            writer.append_data(imageio.imread(buf))
        plt.close(fig)

    if not qa_mode:
        writer.close()
        print(f"✅ Saved {out_gif}")

Rendering 2025-01: 100%|██████████| 744/744 [02:56<00:00,  4.21it/s]


✅ Saved C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\GIFs\2025_01.gif


Rendering 2025-02: 100%|██████████| 658/658 [03:12<00:00,  3.41it/s]


✅ Saved C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\GIFs\2025_02.gif


Rendering 2025-03: 100%|██████████| 744/744 [04:09<00:00,  2.98it/s]


✅ Saved C:\Users\krish\Desktop\SpatialCARE\Hourly\Outputs\GIFs\2025_03.gif
